In [14]:
import numpy as np
import itertools
import random

In [9]:
def heaviside(x):
    return np.array(1 * (x >= 0))
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [131]:
def extension_bool(p,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a vector of shape (L) with elements from {1,0}.  extension[l]=1 iff V[l,:] is in the extension of p
    """
    L = np.shape(V)[0]
    if np.sum(abs(p)) == 0:
        #the all zeros vector
        return np.ones(L)
    else:
        E = np.dot(V,p)/np.sum(abs(p))
        return heaviside(E-1)

def extension_mat(p,V):
    """
    Returns a matrix representing the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        A - A matrix of shape (N,M) with elements from {-1,1}, where N <=L, and the rows of A are the rows of V in the
            extension of p
    """
    
    bool_vec = extension_bool(p,V)
    return V[bool_vec>0,:]
def check_equivalent(p,q,V):
    """
    Checks if p and q have the same extension in V
    
    Inputs:
        p,q - vectors of shape (M,) with elements from {-1,0,1}.  The partially specified feature vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        match - bool, whether or not p and q have the same extension 
    """
    p_bool = extension_bool(p,V)
    q_bool = extension_bool(q,V)
    M = np.shape(q_bool)[0]
    return all([p_bool[i]==q_bool[i] for i in range(M)])

def mutate_p(p,abstraction_set,specification_set):
    """
    Returns q that is equal to p with mutations (specifications and generalizations applied)
    
    Inputs:
        p - the partially specified feature vector to be mutated
        abstraction_set - a set of indices to flip currently specified values of p to zero
        specification_set - a set of tuples (index,value) to change currently unspecified values of p to the given value
    Outputs:
        q - the mutated vector
    """
    
    q = np.copy(p)
    for abstraction in abstraction_set:
        q[abstraction]=0
    for specification_tuple in specification_set:
        q[specification_tuple[0]]=specification_tuple[1]
    return q

def get_allowed_specifications(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, all locations where 
    p can be specified (flipped from a zero to +/- 1) without changing the extension in V.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V -  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        allowed_specifications - a list of tuples of form (index,value) where p[index] can be set to value
        without changing extension
    """
    
    M = np.shape(p)[0]
    
    bool_p = extension_bool(p,V)
    
    A = V[bool_p>0,:]#shape is (N,M)
    N = np.shape(A)[0]
    A_sums = np.sum(A,axis=0)
    
    allowed_specifications = []
    for i in range(M):
        if p[i]==0:
            #we can flip this position to a signed value if only if all A[:,i] or equivalently A_sums[i]== +/- N
            if N==0:
                #empty extension means we can replace all zeros with +1 or -1
                allowed_specifications.append((i,1))
                allowed_specifications.append((i,-1))              
            elif A_sums[i]==N:
                allowed_specifications.append((i,1)) #they're all 1 so we can mutate p[i] to 1
            elif A_sums[i]==-N:
                allowed_specifications.append((i,-1)) #they're all -1 so we can mutate p[i] to -1
    return allowed_specifications

def get_allowed_abstraction_sets(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V -  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        allowed_specification_sets - a list of sets of indices, where each set denotes a set of indices of p
            where a previously specified value can be flipped to zero without changing extension
    """
    bool_p = extension_bool(p,V)
    B = V[bool_p<=0,:]#shape is (T,M), p is not compatible with any vector in B
    T = np.shape(B)[0]
    
    #now we find all subsets J of the specified indices such that if we zeroed out every index of p contained in J, the
    #new vector would be consistent with an element of B
    #this accounts to looking at each element b (row) of B, and finding all the indices where p disagrees 
    #with that element and adding that set of indices to an accumulating set
    forbidden_index_subsets_ = []
    
    for t_val in range(T):
        clash_points = []
        for m_val in range(M):
            if B[t_val,m_val]*p[m_val] == -1:
                clash_points.append(m_val)
        forbidden_index_subsets_.append(frozenset(clash_points))
    forbidden_index_subsets = set(forbidden_index_subsets_)
    
    #B.tolist
    #return zip(forbidden_index_subsets_,B)
    #now we quickly find all 
    
    
def equivalence_class(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        R - a list of vectors where R[i] has shape (M,) and all vectors in R have the same extension in V as p
    """
    
    M = np.shape(p)[0]
    L = np.shape(V)[0]
    V_sums = np.sum(V,axis=0)
    bool_p = extension_bool(p,V)
    
    allowed_specifications = get_allowed_specifications(p,V)
            
    allowed_specification_sets = powerset(allowed_specifications)
    
    #which indices of p can be zeroed out without changing the extension of p in V?
    
    B = V[bool_p<=0,:]#shape is (T,M), p is not compatible with any vector in B
    
    #now we find all subsets q of the specified indices such that if we zeroed out every index of p contained in q, the
    #new vector would be consistent with an element of B
    #this accounts to looking at each element b (row) of B, and finding all the indices where p disagrees 
    #with that element and adding that set of indices to an accumulating set
    forbidden_index_subsets_ = []
    
    for t_val in range(T):
        clash_points = []
        for m_val in range(M):
            if B[t_val,m_val]*p[m_val] == -1:
                clash_points.append(m_val)
        forbidden_index_subsets_.append(frozenset(clash_points))
    forbidden_index_subsets = set(forbidden_index_subsets_)
    
    
    my_queue = [set([i]) for i in prespecified_indices]
    prespecified_index_singletons = [frozenset([i]) for i in prespecified_indices]
    legal_index_sets=[] #will eventually hold all the allowed abstraction sets
    while len(my_queue)>0:
        trial_set = my_queue.pop(0)
        frozen_trial_set = frozenset(trial_set)
        if not (frozen_trial_set in forbidden_index_subsets):
            #it's a legal set of indices to all zero out
            legal_index_sets.append(frozen_trial_set)
            #we also add this set combined with each possible singleton back into the queue for later testing
            for singleton in prespecified_index_singletons:
                if (not singleton.issubset(trial_set)) and (not singleton in forbidden_index_subsets):
                    my_queue.append(trial_set.union(singleton))
    legal_index_sets.append(frozenset())
    
    #Now I must combine the set of allowable specification sets and the set of allowable generalization sets
    #to creat all possible mutated vectors
    
    equivalence_class = []
    for specification_set in allowed_specification_sets:
        for abstraction_set in legal_index_sets:
            q = mutate_p(p,abstraction_set,specification_set)
            equivalence_class.append(q)
    return equivalence_class
    
    #return allowed_specification_sets,legal_index_sets    

In [11]:
def test_eq(p,V,all_possible_p):
    eq_test = equivalence_class(p,V)
    bool_p = extension_bool(p,V)

    match_list = []
    for q in eq_test:
        bool_test = extension_bool(q,V)
        match_list.append(all(bool_p == bool_test))
    print all(match_list)
    eq_set = {tuple(q) for q in eq_test}
    match_list = []
    for q in all_possible_p:
        if not q in eq_set:
            bool_test = extension_bool(np.array(q),V)
            match_list.append(all(bool_p==bool_test))
    print any(match_list)

In [105]:
M = 5
L = 10
all_feature_vecs = [comb for comb in itertools.product([-1,1],repeat=M)]
all_possible_p = [p for p in itertools.product([-1,0,1],repeat=M)]

In [110]:
v_list = random.sample(all_feature_vecs,L)
V = np.stack(v_list,axis=0)
V

array([[-1,  1, -1,  1,  1],
       [ 1,  1,  1, -1,  1],
       [ 1,  1, -1, -1,  1],
       [ 1, -1, -1, -1,  1],
       [-1, -1, -1, -1,  1],
       [ 1,  1, -1, -1, -1],
       [ 1, -1, -1,  1,  1],
       [-1, -1, -1,  1,  1],
       [ 1,  1, -1,  1, -1],
       [ 1, -1,  1,  1, -1]])

In [111]:
p_query = np.array(random.choice(all_possible_p))
p_query

array([ 1,  0,  1, -1, -1])

In [112]:
print(extension_bool(p_query,V))
ext_mat = extension_mat(p_query,V)
ext_mat

[0 0 0 0 0 0 0 0 0 0]


array([], shape=(0, 5), dtype=int64)

In [132]:
get_allowed_abstraction_sets(p_query,V)

[(frozenset({0, 2, 3, 4}), array([-1,  1, -1,  1,  1])),
 (frozenset({4}), array([ 1,  1,  1, -1,  1])),
 (frozenset({2, 4}), array([ 1,  1, -1, -1,  1])),
 (frozenset({2, 4}), array([ 1, -1, -1, -1,  1])),
 (frozenset({0, 2, 4}), array([-1, -1, -1, -1,  1])),
 (frozenset({2}), array([ 1,  1, -1, -1, -1])),
 (frozenset({2, 3, 4}), array([ 1, -1, -1,  1,  1])),
 (frozenset({0, 2, 3, 4}), array([-1, -1, -1,  1,  1])),
 (frozenset({2, 3}), array([ 1,  1, -1,  1, -1])),
 (frozenset({3}), array([ 1, -1,  1,  1, -1]))]